In [ ]:
setwd("bkp/output-www1/")

In [ ]:
library(jsonlite)
library(writexl)
library(readxl)
library(dplyr)
library(car)

First we need to import the data and tidy up the datatypes a bit

In [ ]:
datafile <- file("test.json", open="r")
data <- stream_in(datafile, flatten=TRUE)
close(datafile)
data$treatment.ethnicity <- as.factor(data$treatment.ethnicity)
data$treatment.gender <- as.factor(data$treatment.gender)
data$type <- as.factor(data$type)
data$scraper <- as.factor(data$scraper)
data$block_id <- as.factor(data$block_id)
data$agent_id <- as.factor(data$agent_id)

Now we split off just the ads from our data

In [ ]:
ads <- data[data$type == 'ad', ]
ads$ranking <- NULL

Next we write out the data for coding

In [ ]:
unique_ads <- unique(ads[c("ad.title", "ad.url")])
write_xlsx(unique_ads, "unique-ads.xlsx")

Now code the results, read them back in, and join them to the existing data

In [ ]:
categorized_ads <- read_excel("unique-ads-coded.xlsx")
categorized_ads$ad.category <- as.factor(categorized_ads$ad.category)
categorized_ads$ad.title <- NULL
merged <- merge(x=ads, y=categorized_ads)

In [ ]:
aggregator_ads <- count(subset(merged, ad.category == "aggregator"), agent_id, treatment.gender, treatment.ethnicity, scraper)
res.aggregator <- aov(n ~ treatment.ethnicity * treatment.gender, data = aggregator_ads)
summary(res.aggregator)